In [1]:
from agent import Agent
from referee.game import PlayerColor
agent = Agent(PlayerColor.RED)

agent

Testing: I am playing as red


In [2]:
action = agent.action()
action

SpawnAction(cell=HexPos(r=4, q=6))

In [3]:
from agent.board import MatrixBoard
import numpy as np

matrix = np.zeros([2,7,7], dtype=int)
board = MatrixBoard(matrix, 0, 0, 0)

In [4]:
from referee.game import \
    PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir

board.apply_action(SpawnAction(HexPos(1,1)), PlayerColor.RED)
board.apply_action(SpawnAction(HexPos(1,2)), PlayerColor.BLUE)
board.apply_action(SpreadAction(HexPos(1,1), HexDir.UpRight), PlayerColor.RED)

In [5]:
print(board.render(use_color=True, use_unicode=True))

                         ..     
                     ..      ..     
                 ..      ..      ..     
             ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
     ..      r1      ..      ..      ..      ..     
 ..      ..      ..      ..      ..      ..      ..     
     ..      b1      ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
             ..      ..      ..      ..     
                 ..      ..      ..     
                     ..      ..     
                         ..     



In [6]:
import math
from enum import Enum
from referee.game import \
    PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir
from agent.board import MatrixBoard

class MCState(Enum):
    UNVISITIED = 0;
    UNEXPANDED = 1;
    EXPANDED = 2;

class MCNode:
    def __init__(self, 
                 time_limit: float | None, 
                 space_limit: float | None,
                 board: MatrixBoard, 
                 color: PlayerColor,
                 action: Action = None,
                 parent = None,
                 ) -> None:
        self.playouts = 0
        self.wins = 0
        self.state = MCState.UNVISITIED
        self.time_limit = time_limit
        self.space_limit = space_limit
        self.board = board
        self.parent = parent
        self.children = []
        self.action = action
        self.color = color
        # self.ubc
    
    def is_over(self) -> bool:
        return self.board.game_over()
    
    def winner_color(self) -> PlayerColor | None: 
        return self.board.winner_color()
    
    def _ucb(self, c):
        '''
        Calculate the UCB score.
        '''
        return self.wins / self.playouts \
                + c * math.sqrt(2 * math.log(self.playouts) / self.playouts)

In [7]:
state_prev = np.zeros([2,7,7], dtype=int)
board_prev = MatrixBoard(state_prev, 0, 0, 0)
node_prev = MCNode(0, 0, board_prev, PlayerColor.RED, None, None)

node_curr = MCNode(0, 0, board_prev, PlayerColor.RED, None, None)

In [8]:
import numpy as np

# Create a NumPy array
# my_array = np.array([[1, 2, 3], [4, 5, 6]])
my_array = np.zeros([2,2,7,7], dtype=int).reshape([2, 2*7*7])

# Open the file in append mode
with open('my_file.csv', mode='w') as file:
    # Use np.savetxt() with the file object
    np.savetxt(file, my_array, delimiter=',')


In [9]:
# Read the CSV file as a float data type
my_array = np.loadtxt('my_file.csv', delimiter=',', dtype=float)
my_array.shape

(2, 98)

In [10]:
my_array.reshape([2,2,7,7])

array([[[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0.,

In [11]:
my_array.shape

(2, 98)

In [12]:
def convert_action_to_array(action: Action) -> np.ndarray:
    match action:
        case SpawnAction(cell):
            return np.array([cell.r, cell.q, 0, 0])
            # pass
        case SpreadAction(cell, direction):
            return np.array([cell.r, cell.q, direction.r, direction.q])

In [13]:
convert_action_to_array(SpreadAction(HexPos(1,1), HexDir.Down)).reshape([1,-1]).shape

(1, 4)

In [14]:
i = 207
np.array([i], dtype=int).reshape(1, -1)

array([[207]])

In [15]:
board = np.zeros([2,7,7], dtype=int)
board[0,1,1] = 3
board[1,3,4] = 5

# translate the board into state 
new_board = np.zeros([3,7,7], dtype=int)
new_board[-1] = np.ones([1,7,7], dtype=int)

# Channel 1: represents the positions of tokens
set_oppo = lambda x: -1 if x > 0 else 0
set_curr = lambda x: 1 if x > 0 else 0
oppo = np.array([[set_oppo(e) for e in row] for row in board[1]])
curr = np.array([[set_curr(e) for e in row] for row in board[0]])
new_board[0] = oppo + curr

# Channel 2: represents the power of all tokens
add = board[0] + board[1]
new_board[1] = np.copy(add)

In [16]:
new_board

array([[[ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, -1,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  3,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  5,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0]],

       [[ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1]]])

In [17]:
import torch
import numpy as np

class ExampleModel(torch.nn.Module):
    def __init__(self):
        super(ExampleModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(2, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(32 * 7 * 7, 128)
        self.relu3 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(128, 1)
        self.tanh = torch.nn.Tanh()

    def forward(self, x):
        x = torch.from_numpy(x).float() # convert input numpy array to torch tensor
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = x.view(-1, 32 * 7 * 7) # flatten the tensor
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.tanh(x)
        return x

model = ExampleModel()
input_array = np.random.rand(2, 7, 7)
output = model(input_array)
print(output)


tensor([[-0.0594]], grad_fn=<TanhBackward0>)


In [18]:
# import numpy as np

# import numpy as np

# def example_model_predict(input_array):
#     # define the weights and biases for each layer of the neural network
#     conv1_weight = np.random.rand(16, 2, 3, 3)
#     conv1_bias = np.random.rand(16)
#     conv2_weight = np.random.rand(32, 16, 3, 3)
#     conv2_bias = np.random.rand(32)
#     fc1_weight = np.random.rand(128, 32 * 7 * 7)
#     fc1_bias = np.random.rand(128)
#     fc2_weight = np.random.rand(1, 128)
#     fc2_bias = np.random.rand(1)

#     # perform the operations in the neural network manually
#     x = input_array.astype(np.float32)
#     x = np.transpose(x, (0, 3, 1, 2))  # transpose to (batch_size, channels, height, width)
#     x = np.apply_along_axis(lambda m: np.convolve(m, conv1_weight.T.flatten(), mode='same') + conv1_bias, axis=(2, 3), arr=x)
#     x = np.maximum(0, x)
#     x = np.apply_along_axis(lambda m: np.convolve(m, conv2_weight.T.flatten(), mode='same') + conv2_bias, axis=(2, 3), arr=x)
#     x = np.maximum(0, x)
#     x = x.reshape(-1, 32 * 7 * 7)
#     x = np.dot(x, fc1_weight.T) + fc1_bias
#     x = np.maximum(0, x)
#     x = np.dot(x, fc2_weight.T) + fc2_bias
#     x = np.tanh(x)

#     # return the final output
#     return x.item()

# # create a random input array with shape (2, 7, 7)
# input_array = np.random.rand(2, 7, 7)

# # reshape the input array to have shape (2, 1, 7, 7) to match the expected input shape
# input_array = input_array.reshape((2, 1, 7, 7))

# # call the example_model_predict function with the reshaped input array
# output = example_model_predict(input_array)

# # print the output
# print(output)


In [19]:
import os

for file in os.listdir('/Users/clarec/Desktop/COMP30024-AI-ProjectB/agent/log/'):
    print(file)
# LOG_PATH = '/Users/clarec/Desktop/COMP30024-AI-ProjectB/agent/log/random_random_'

# np1 = np.load(LOG_PATH+)

rr_5_R.csv
rr_17.csv
rr_3_R.csv
rr_1_R.csv
rr_7_B.csv
rr_2_B.csv
rr_4_R.csv
rr_6_B.csv
rr_10_R.csv
rr_8_R.csv
rr_14_R.csv
rr_12_B.csv
rr_16_D.csv
rr_17_B.csv
rr_13_R.csv
rr_11_B.csv
rr_15_R.csv
rr_9_R.csv


In [20]:
LOG_RANDOM_PATH = '/Users/clarec/Desktop/COMP30024-AI-ProjectB/agent/log/rr_1_R.csv'

X = np.loadtxt(LOG_RANDOM_PATH, delimiter=',', dtype=float)


In [21]:
X[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
y = np.ones([343, 1])


In [43]:
player = 1
def convert_to_state_input(board):
    ''' Translate the board into state '''
    global player
    
    # if len(board.shape) == 0:
    #     print(board)
    #     return np.array([])
    
    if len(board.shape) < 3:
        board = board.reshape([2, 7, 7])
        
    # n = board.shape[0]
    new_board = np.zeros([3,7,7], dtype=int)
    # new_board[-1] = np.ones([player,7,7], dtype=int)

    # Channel 1: represents the positions of tokens
    set_oppo = lambda x: -1 if x > 0 else 0
    set_curr = lambda x: 1 if x > 0 else 0
    oppo = np.array([[set_oppo(e) for e in r] for r in board[1]])
    curr = np.array([[set_curr(e) for e in r] for r in board[0]])
    new_board[0] = oppo + curr

    # Channel 2: represents the power of all tokens
    add = board[0] + board[1]
    new_board[1] = np.copy(add)
    
    # Channel 3: Which color is playing (RED: 1 BLUE: -1)
    if player == 1:
        new_board[-1] = np.ones([1,7,7], dtype=int)
    else:
        new_board[-1] = np.full(fill_value=player, shape=[1,7,7], dtype=int)
    
    player = -player 
    
    return new_board

res = np.array([convert_to_state_input(xi) for xi in X])
print(res.shape)
player = 1

(343, 3, 7, 7)


In [44]:
import os

player = 1
PATH = '/Users/clarec/Desktop/COMP30024-AI-ProjectB/agent/log/'
read_input = np.array([]).reshape([-1, 3, 7, 7])
target_arr = np.array([]).reshape([-1, 1])
# np.concatenate([read_input, res]).shape
for file in os.listdir(PATH):
    print(file)
    tmp = np.loadtxt(PATH + file, delimiter=',', dtype=float)
    tmp = np.array([convert_to_state_input(xi) for xi in tmp])
    read_input = np.concatenate([read_input, tmp])
    
    # determine which player has won
    if file[-4] =="R":
        win = 1
    elif file[-4] =="B":
        win = -1 
    else:  # draw
        win = 0
    
    win_score = np.full(shape=[tmp.shape[0], 1], fill_value=win)
    target_arr = np.concatenate([target_arr, win_score])

rr_5_R.csv
rr_3_R.csv
rr_1_R.csv
rr_7_B.csv
rr_2_B.csv
rr_4_R.csv
rr_6_B.csv
rr_10_R.csv
rr_8_R.csv
rr_14_R.csv
rr_12_B.csv
rr_16_D.csv
rr_17_B.csv
rr_13_R.csv
rr_11_B.csv
rr_15_R.csv
rr_9_R.csv


In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.tanh(x)
        return x



In [48]:
# Convert input numpy array to PyTorch tensor
input_array = read_input
input_tensor = torch.from_numpy(input_array).float()
target_array = target_arr
target = torch.from_numpy(target_array).float()

# Create an instance of the model
model = CNNModel()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(input_tensor)
    # print(output.shape)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Make predictions
model.eval()
with torch.no_grad():
    output = model(input_tensor)
    predictions = output.numpy()

# Print the predicted values
print('Predictions:', predictions)

Epoch [1/10], Loss: 0.0198
Epoch [2/10], Loss: 0.0185
Epoch [3/10], Loss: 0.0172
Epoch [4/10], Loss: 0.0160
Epoch [5/10], Loss: 0.0149
Epoch [6/10], Loss: 0.0138
Epoch [7/10], Loss: 0.0128
Epoch [8/10], Loss: 0.0118
Epoch [9/10], Loss: 0.0109
Epoch [10/10], Loss: 0.0100
Predictions: [[0.11007989]
 [0.10057984]
 [0.10938258]
 ...
 [0.10628287]
 [0.07092883]
 [0.09704246]]


In [ ]:
predictions

array([[0.8891705 ],
       [0.89073676],
       [0.9308712 ],
       [0.9264568 ],
       [0.933289  ],
       [0.95436734],
       [0.960326  ],
       [0.9655226 ],
       [0.97077554],
       [0.9662922 ],
       [0.9765695 ],
       [0.97580016],
       [0.9775205 ],
       [0.97680503],
       [0.9770405 ],
       [0.9778798 ],
       [0.97712314],
       [0.9817124 ],
       [0.98355615],
       [0.9883012 ],
       [0.98813343],
       [0.9895682 ],
       [0.99040353],
       [0.99042726],
       [0.98989874],
       [0.9906    ],
       [0.991933  ],
       [0.99176204],
       [0.9924534 ],
       [0.9912353 ],
       [0.9917103 ],
       [0.9902898 ],
       [0.9921826 ],
       [0.9908978 ],
       [0.99203616],
       [0.9918618 ],
       [0.9925237 ],
       [0.99260527],
       [0.99310434],
       [0.9927216 ],
       [0.9931453 ],
       [0.9940703 ],
       [0.99473435],
       [0.99541175],
       [0.9959474 ],
       [0.9955719 ],
       [0.99513423],
       [0.994